In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa

%load_ext autoreload
%autoreload 2

In [2]:
from multichannel_spectrograms import *

In [387]:
win_length = 512
hop_length = 64

In [388]:
np.log10(0.5)

In [389]:
is_odd = win_length % 2 == 1
lin_spacing = 1 / win_length
idx0 = win_length // 2
lin_freqs = np.concatenate([
    np.linspace(-0.5, 0, win_length // 2 + 1, endpoint=True),
    np.linspace(lin_spacing, 0.5, (win_length - 1) // 2, endpoint=is_odd)
])
neg_log_freqs = [-(10 ** i) for i in np.linspace(-0.3010299956639812, -3, win_length // 2, endpoint=True)]
log_freqs = np.array(neg_log_freqs + [0] + [-f for f in neg_log_freqs[not is_odd:][::-1]])
lr_freqs = librosa.fft_frequencies(sr=1, n_fft=win_length)
print(idx0)

In [390]:
plt.plot(lin_freqs)
plt.plot(log_freqs)
plt.plot(lr_freqs, ':')
plt.plot([0, win_length], [0.5, 0.5], 'k--')
print(lin_freqs.shape, log_freqs.shape, lr_freqs.shape)
print(lin_freqs[-1], log_freqs[-1], lr_freqs[-1])

In [391]:
plt.plot(lin_freqs[idx0:])
plt.plot(log_freqs[idx0:])
plt.plot(lr_freqs, ':')
plt.yscale('log')
print(lin_freqs[idx0], log_freqs[idx0], lr_freqs[0])

In [392]:
plt.plot(lin_freqs[idx0:])
plt.plot(-lin_freqs[:idx0:-1])
plt.plot(log_freqs[idx0:])
plt.plot(-log_freqs[:idx0:-1])
plt.plot(lr_freqs, ':')
plt.yscale('log')
print(lin_freqs[idx0], log_freqs[idx0], lr_freqs[0])

In [393]:
omega = np.exp(-2j * np.pi / win_length)
Wlin = omega ** (np.outer(np.fft.ifftshift(lin_freqs) * win_length, np.arange(win_length))) #/ np.sqrt(win_length)
Wlog = omega ** (np.outer(np.fft.ifftshift(log_freqs) * win_length, np.arange(win_length))) #/ np.sqrt(win_length)

In [394]:
plt.imshow(Wlin.real)
plt.colorbar()

In [395]:
plt.imshow(Wlog.real)
plt.colorbar()

In [396]:
plt.imshow(np.abs(Wlin @ np.conjugate(Wlin.T)))
plt.colorbar()

In [397]:
plt.imshow(np.angle(Wlin), aspect='equal')

In [398]:
plt.imshow(np.angle(Wlog), aspect='equal')

In [399]:
x = np.sin(2 * np.pi * 50 * np.arange(win_length) / win_length) + 0.5
plt.plot(x)

In [400]:
plt.plot(np.hanning(win_length))
plt.plot(np.abs(np.fft.fftshift(np.fft.fft(np.hanning(win_length))) / np.sqrt(win_length)))

In [401]:
window = np.hanning(win_length)
Xlin = Wlin @ (window * x)
Xlog = Wlog @ (window * x)
Xfft = np.fft.fft(window * x)
aXlin = np.fft.fftshift(np.abs(Xlin))
aXlog = np.fft.fftshift(np.abs(Xlog))
aXfft = np.fft.fftshift(np.abs(Xfft))
plt.plot(lin_freqs, aXlin)
plt.plot(log_freqs, aXlog)
plt.plot(lin_freqs, aXfft, ':')

In [402]:
is_even = not is_odd
split_idx = win_length // 2 + is_odd
rXlin = Xlin[:split_idx]
rXlin[1:] += Xlin[split_idx + is_even:][::-1].conjugate()
arXlin = np.abs(rXlin)
rXlog = Xlog[:split_idx]
rXlog[1:] += Xlog[split_idx + is_even:][::-1].conjugate()
arXlog = np.abs(rXlog)
rXfft = Xfft[:split_idx]
rXfft[1:] += Xfft[split_idx + is_even:][::-1].conjugate()
arXfft = np.abs(rXfft)
plt.plot(lin_freqs[idx0:], arXlin)
plt.plot(log_freqs[idx0:], arXlog)
plt.plot(lin_freqs[idx0:], arXfft, ':')

In [406]:
plt.plot(np.arange(split_idx), np.abs(Xlin[:split_idx]))
plt.plot(np.arange(1, split_idx), np.abs(Xlin[split_idx + is_even:][::-1]))

In [407]:
plt.plot(np.abs(Xlog[:split_idx]))